Here, in line-up prediction we cannot use a whole year as the test set. Since many changes have been occured (data shift), we have many new players this year and many retired.

In [12]:
import pandas as pd
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import os
#from sklearn.preprocessing import OneHotEncoder
"""from google.colab import drive


drive.mount('/content/drive')
ds_path = '/content/drive/MyDrive/Data_Mining_Project/Datasets/'"""

ds_path = 'Datasets/'
assert os.path.exists(ds_path)

features = ['outcome', 'season', 'home_team','away_team','starting_min','end_min','home_0','home_1','home_2','home_3','home_4','away_0','away_1','away_2','away_3','away_4']
home_players_columns = ['home_0','home_1','home_2','home_3','home_4']
away_players_columns = ['away_0','away_1','away_2','away_3','away_4']


df = pd.DataFrame()
# Load data from matchups-2007.csv to matchups-2012.csv and append them to df
for i in range(2007, 2013):
    df1 = pd.read_csv(ds_path + "matchups-" + str(i) + ".csv")[features]
    print(len(df1))
    df = pd.concat([df, df1])
df.reset_index(drop=True, inplace=True)
print(f"len of final df: {len(df)}")
print("first print \n" + str(df))
print(df.shape)


raw_df = df.copy()


27500
26593
26407
26344
26447
21241
len of final df: 154532
first print 
        outcome  season home_team away_team  starting_min  end_min  \
0            -1    2007       LAL       PHO             0        5   
1            -1    2007       LAL       PHO             6        7   
2             1    2007       LAL       PHO             8        9   
3             1    2007       LAL       PHO            10       10   
4            -1    2007       LAL       PHO            11       11   
...         ...     ...       ...       ...           ...      ...   
154527       -1    2012       CHA       NOH            38       39   
154528       -1    2012       CHA       NOH            40       41   
154529       -1    2012       CHA       NOH            42       42   
154530       -1    2012       CHA       NOH            43       45   
154531       -1    2012       CHA       NOH            46       47   

                 home_0            home_1            home_2            home_3  \
0    

In [13]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Model

# Label encoding for teams and players
player_le = LabelEncoder()
team_le = LabelEncoder()
season_le = LabelEncoder()

# Encoding players
# Flatten the DataFrame to encode all player positions together
player_columns = [f'home_{i}' for i in range(5)] + [f'away_{i}' for i in range(5)]
all_players = raw_df[player_columns].values.flatten()

# Fit the LabelEncoder on all player names
player_le.fit(all_players)

# Transform each player column in the DataFrame
for col in player_columns:
    df[col] = player_le.transform(raw_df[col])


# Encoding teams
df['home_team'] = team_le.fit_transform(raw_df['home_team'])
df['away_team'] = team_le.transform(raw_df['away_team'])
df['season'] = season_le.fit_transform(raw_df['season'])

print(df)

        outcome  season  home_team  away_team  starting_min  end_min  home_0  \
0            -1       0         12         24             0        5      41   
1            -1       0         12         24             6        7      41   
2             1       0         12         24             8        9     482   
3             1       0         12         24            10       10     482   
4            -1       0         12         24            11       11     511   
...         ...     ...        ...        ...           ...      ...     ...   
154527       -1       5          2         18            38       39      72   
154528       -1       5          2         18            40       41      72   
154529       -1       5          2         18            42       42      72   
154530       -1       5          2         18            43       45      72   
154531       -1       5          2         18            46       47      72   

        home_1  home_2  home_3  home_4 

In [7]:
"""team_season_players = {}

# Process home team players
for _, row in raw_df.iterrows():
    # Home team
    home_team = row['home_team']
    season = row['season']
    home_players = {row[f'home_{i}'] for i in range(5)}
    
    if home_team not in team_season_players:
        team_season_players[home_team] = {}
    if season not in team_season_players[home_team]:
        team_season_players[home_team][season] = set()
    
    team_season_players[home_team][season].update(home_players)
    
    # Away team (repeating the process for the away team)
    away_team = row['away_team']
    away_players = {row[f'away_{i}'] for i in range(5)}
    
    if away_team not in team_season_players:
        team_season_players[away_team] = {}
    if season not in team_season_players[away_team]:
        team_season_players[away_team][season] = set()
    
    team_season_players[away_team][season].update(away_players)"""

In [41]:
# OTHER
import pandas as pd
import numpy as np

# Assuming df is your DataFrame with the player columns
# Example columns: ['home_0', 'home_1', ..., 'away_4']

"""def preprocess_data(df1):
    # Initialize new columns
    df = df1.copy()
    df['target_player_name'] = None
    df['player_names'] = None
    
    
    for index, row in df.iterrows():
        # Get all player names for the row
        players = row[['home_0', 'home_1', 'home_2', 'home_3', 'home_4', 'away_0', 'away_1', 'away_2', 'away_3', 'away_4']].tolist()
        
        # Randomly select one player as the target
        target_player = np.random.choice(players)
        
        # Use the remaining players as the context
        context_players = [player for player in players if player != target_player]
        
        # Assign to the DataFrame
        df.at[index, 'target_player_name'] = target_player
        df.at[index, 'player_names'] = context_players
        
    
    df = df.drop(['home_0', 'home_1', 'home_2', 'home_3', 'home_4', 'away_0', 'away_1', 'away_2', 'away_3', 'away_4'], axis=1)
    return df

"""


def preprocess_data_expanded(df1):
    # Initialize a list to hold new rows
    new_rows = []
    
    for index, row in df1.iterrows():
        # Get all player names for the row
        players = row[['home_0', 'home_1', 'home_2', 'home_3', 'home_4', 'away_0', 'away_1', 'away_2', 'away_3', 'away_4']].tolist()
        
        # Iterate over all players, selecting each one as the target once
        for target_player in players:
            # Use the remaining players as the context
            context_players = [player for player in players if player != target_player]
            
            # Create a new row with this player as the target
            new_row = row.copy()
            new_row['target_player_name'] = target_player
            new_row['player_names'] = context_players
            
            # Append the new row to the list
            new_rows.append(new_row)
    
    # Create a new DataFrame from the list of new rows
    new_df = pd.DataFrame(new_rows).reset_index(drop=True)
    
    # Drop the original player columns
    new_df = new_df.drop(['home_0', 'home_1', 'home_2', 'home_3', 'home_4', 'away_0', 'away_1', 'away_2', 'away_3', 'away_4'], axis=1)
    return new_df


# Apply preprocessing
df_preprocessed = preprocess_data(df)

In [42]:
# Flatten the DataFrame slice into a single array of player names
player_names_array = df[["home_0", "home_1", "home_2", "home_3", "home_4", "away_0", "away_1", "away_2", "away_3", "away_4"]].values.flatten()

# Find the unique player names in the array
unique_player_names = np.unique(player_names_array)
print(len(unique_player_names))


unique_teams = np.unique(df[['home_team', 'away_team']].values.flatten())

unique_seasons = np.unique(df['season'].values)
unique_seasons = np.array([str(season) for season in unique_seasons])


812


In [43]:
# OTHER
print(df_preprocessed)
print(df_preprocessed.loc[1,'player_names'])
print(len(unique_player_names))

        outcome  season  home_team  away_team  starting_min  end_min  \
0            -1       0         12         24             0        5   
1            -1       0         12         24             6        7   
2             1       0         12         24             8        9   
3             1       0         12         24            10       10   
4            -1       0         12         24            11       11   
...         ...     ...        ...        ...           ...      ...   
154527       -1       5          2         18            38       39   
154528       -1       5          2         18            40       41   
154529       -1       5          2         18            42       42   
154530       -1       5          2         18            43       45   
154531       -1       5          2         18            46       47   

       target_player_name                                  player_names  
0                      41  [482, 511, 692, 715, 81, 474, 634,

In [45]:
# data split
X_train_bu, X_test_bu = train_test_split(df_preprocessed, test_size=0.2, random_state=52)
print(X_train_bu)
print(X_test_bu)

        outcome  season  home_team  away_team  starting_min  end_min  \
120762       -1       4         16         31            13       15   
80136        -1       2         13         18            24       24   
104090        1       3         23         25            23       24   
24772        -1       0         19          7            42       44   
43933        -1       1         20          5            21       21   
...         ...     ...        ...        ...           ...      ...   
58268        -1       2         12         18            17       17   
129110        1       4         11         25             0        3   
69783         1       2          1         26            46       47   
56845        -1       2          8         21            21       21   
42652         1       1         29          1            46       47   

       target_player_name                                   player_names  
120762                791    [53, 413, 546, 596, 12, 330, 46

In [52]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization

def select_model(num):
    global model
    if num == 1:
        player_input_dim = len(player_le.classes_)

        player_input = Input(shape=(9,), dtype='int32', name='player_input')
        player_embedding = Embedding(input_dim=player_input_dim + 1, output_dim=64)(player_input)
        player_embedding_pooled = GlobalAveragePooling1D()(player_embedding)
        
        dense_layer = Dense(256, activation='relu')(player_embedding_pooled)
        dense_layer = Dense(256, activation='relu')(dense_layer)
        output_layer = Dense(player_input_dim, activation='softmax')(dense_layer)
        
        model = Model(inputs=player_input, outputs=output_layer)
    elif num == 2:
        player_input_dim = len(player_le.classes_)

        player_input = Input(shape=(9,), dtype='int32', name='player_input')
        player_embedding = Embedding(input_dim=player_input_dim + 1, output_dim=64)(player_input)
        player_embedding_pooled = GlobalAveragePooling1D()(player_embedding)
        
        dense_layer = Dense(512, activation='relu')(player_embedding_pooled)
        dense_layer = Dense(256, activation='relu')(dense_layer)
        output_layer = Dense(player_input_dim, activation='softmax')(dense_layer)
        
        model = Model(inputs=player_input, outputs=output_layer)

    elif num == 3:
        player_input_dim = len(player_le.classes_)

        player_input = Input(shape=(9,), dtype='int32', name='player_input')
        player_embedding = Embedding(input_dim=player_input_dim + 1, output_dim=64)(player_input)
        player_embedding_pooled = GlobalAveragePooling1D()(player_embedding)
        
        dense_layer = Dense(128, activation='relu')(player_embedding_pooled)
        dense_layer = Dense(128, activation='relu')(dense_layer)
        dense_layer = Dense(128, activation='relu')(dense_layer)
        output_layer = Dense(player_input_dim, activation='softmax')(dense_layer)
        
        model = Model(inputs=player_input, outputs=output_layer)
    #elif num == 2:
        

In [47]:
epochs = 20

def lr_schedule(epoch):
    """Learning rate schedule function."""
    initial_lr = 0.001
    if epoch < epochs / 4:
        return initial_lr
    else:
        return initial_lr * np.exp(-0.1 * epoch)

In [49]:
#only players
from tensorflow.keras.layers import Input, Embedding, concatenate, Dropout, Dense, GlobalAveragePooling1D, Reshape, Flatten
from tensorflow.keras.callbacks import TensorBoard, LearningRateScheduler
import shutil
from numpy.random import seed

res_path = 'Results2/'
def train_NN(mod_num):
    accuracy_list = []
    
    for a_seed in range(100, 111):
        np.random.seed(a_seed)
        seed(a_seed)
        select_model(mod_num)
        
        X_train, X_val = train_test_split(X_train_bu, test_size=0.2, random_state=a_seed)
        
        lr_scheduler = LearningRateScheduler(lr_schedule)
        
        # Compile the model
        model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
        
        player_names_padded_train = pad_sequences(X_train['player_names'], maxlen=9, padding='post', value=0)
        target_player_names_train = tf.cast(np.array(X_train['target_player_name'].values).astype(np.float32), dtype=tf.int32)
        player_names_padded_val = pad_sequences(X_val['player_names'], maxlen=9, padding='post', value=0)
        target_player_names_val = tf.cast(np.array(X_val['target_player_name'].values).astype(np.float32), dtype=tf.int32)
    
        # Train the model
        history = model.fit(player_names_padded_train, target_player_names_train, epochs=epochs, 
                            validation_data=(player_names_padded_val, target_player_names_val), verbose=False,
                            callbacks=[lr_scheduler])

        loss, accuracy = model.evaluate(player_names_padded_val, target_player_names_val)
        accuracy_list.append(accuracy)


    
    
    # Retrain and Evaluate the model
    # Clear log directory if it exists
    log_dir = "logs23_ModelNum" + str(mod_num) + "/fit/"
    if os.path.exists(log_dir):
        shutil.rmtree(log_dir)
    os.makedirs(log_dir)
    sm_checkpoint = tf.keras.callbacks.ModelCheckpoint(res_path + 'checkpoints/best_model_' + str(mod_num) + '.h5', 
                                							monitor='val_accuracy', verbose=0, 
                                							save_best_only=True, mode='max')
    # Define TensorBoard callback
    tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)
    select_model(mod_num)
    lr_scheduler = LearningRateScheduler(lr_schedule)
    
    model.compile(optimizer='adam',
                      loss='sparse_categorical_crossentropy',
                      metrics=['accuracy'])

    model.summary()
    
    player_names_padded = pad_sequences(X_train_bu['player_names'], maxlen=9, padding='post', value=0)
    target_player_names = tf.cast(np.array(X_train_bu['target_player_name'].values).astype(np.float32), dtype=tf.int32)
    test_player_names_padded = pad_sequences(X_test['player_names'], maxlen=9, padding='post', value=0)
    test_target_player_names = tf.cast(np.array(X_test['target_player_name'].values).astype(np.float32), dtype=tf.int32)
    
    history = model.fit(player_names_padded, target_player_names, epochs=epochs, validation_data=(test_player_names_padded, test_target_player_names), 
                        verbose=False,
                        callbacks=[tensorboard_callback, lr_scheduler, sm_checkpoint])

    loss, accuracy = model.evaluate(test_player_names_padded, test_target_player_names)
    
    print(f'Test Loss: {loss:.4f}')
    print(f'Test Accuracy: {accuracy:.4f}')
    
    print()
    print("From Cross_validation:")
    print(f"Accuracy: {np.mean(accuracy_list):.4f}, std: {np.std(accuracy_list):.4f}")
    
    







"""log_dir = "logs23_ModelNum1/fit/"
if os.path.exists(log_dir):
    shutil.rmtree(log_dir)
os.makedirs(log_dir)
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)
lr_scheduler = LearningRateScheduler(lr_schedule)
select_model(1)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Assuming X_train['player_names'] and X_train['target_player_name'] are defined correctly
player_names_padded = pad_sequences(X_train['player_names'], maxlen=9, padding='post', value=0)
target_player_names = tf.cast(np.array(X_train['target_player_name'].values).astype(np.float32), dtype=tf.int32)





model.fit(player_names_padded, 
          target_player_names,
          batch_size=32, 
          epochs=epochs, 
          validation_split=0.2,
          verbose=False,
          callbacks=[tensorboard_callback, lr_scheduler])


test_player_names_padded = pad_sequences(X_test['player_names'], maxlen=9, padding='post', value=0)
test_target_player_names = tf.cast(np.array(X_test['target_player_name'].values).astype(np.float32), dtype=tf.int32)
loss, accuracy = model.evaluate(test_player_names_padded, test_target_player_names)
print(accuracy)
"""

'log_dir = "logs23_ModelNum1/fit/"\nif os.path.exists(log_dir):\n    shutil.rmtree(log_dir)\nos.makedirs(log_dir)\ntensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)\nlr_scheduler = LearningRateScheduler(lr_schedule)\nselect_model(1)\n\nmodel.compile(optimizer=\'adam\', loss=\'sparse_categorical_crossentropy\', metrics=[\'accuracy\'])\n\n# Assuming X_train[\'player_names\'] and X_train[\'target_player_name\'] are defined correctly\nplayer_names_padded = pad_sequences(X_train[\'player_names\'], maxlen=9, padding=\'post\', value=0)\ntarget_player_names = tf.cast(np.array(X_train[\'target_player_name\'].values).astype(np.float32), dtype=tf.int32)\n\n\n\n\n\nmodel.fit(player_names_padded, \n          target_player_names,\n          batch_size=32, \n          epochs=epochs, \n          validation_split=0.2,\n          verbose=False,\n          callbacks=[tensorboard_callback, lr_scheduler])\n\n\ntest_player_names_padded = pad_sequences(X_test[\'player_names\'], maxlen=9, 

In [50]:
train_NN(1)

773/773 [==============================] - 1s 1ms/step - loss: 3.6333 - accuracy: 0.2066
Model: "model_31"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 player_input (InputLayer)   [(None, 9)]               0         
                                                                 
 embedding_33 (Embedding)    (None, 9, 64)             52032     
                                                                 
 global_average_pooling1d_2  (None, 64)                0         
 9 (GlobalAveragePooling1D)                                      
                                                                 
 dense_91 (Dense)            (None, 256)               16640     
                                                                 
 dense_92 (Dense)            (None, 256)               65792     
                                                                 
 dense_93 (Dense)            (None,

C:\Python-3.11.4\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


966/966 [==============================] - 1s 1ms/step - loss: 3.3283 - accuracy: 0.2391
Test Loss: 3.3283
Test Accuracy: 0.2391

From Cross_validation:
Accuracy: 0.2102, std: 0.0027


In [51]:
train_NN(2)

773/773 [==============================] - 1s 1ms/step - loss: 3.5286 - accuracy: 0.2140
Model: "model_43"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 player_input (InputLayer)   [(None, 9)]               0         
                                                                 
 embedding_45 (Embedding)    (None, 9, 64)             52032     
                                                                 
 global_average_pooling1d_4  (None, 64)                0         
 1 (GlobalAveragePooling1D)                                      
                                                                 
 dense_127 (Dense)           (None, 512)               33280     
                                                                 
 dense_128 (Dense)           (None, 256)               131328    
                                                                 
 dense_129 (Dense)           (None,

In [53]:
train_NN(3)

773/773 [==============================] - 1s 984us/step - loss: 3.7629 - accuracy: 0.1596
Model: "model_55"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 player_input (InputLayer)   [(None, 9)]               0         
                                                                 
 embedding_57 (Embedding)    (None, 9, 64)             52032     
                                                                 
 global_average_pooling1d_5  (None, 64)                0         
 3 (GlobalAveragePooling1D)                                      
                                                                 
 dense_174 (Dense)           (None, 128)               8320      
                                                                 
 dense_175 (Dense)           (None, 128)               16512     
                                                                 
 dense_176 (Dense)           (Non

In [23]:
# Ensure it's padded to have a fixed length if it's not already
from tensorflow.keras.preprocessing.sequence import pad_sequences

"""target_player_names = tf.cast(np.array(X_train['target_player_name'].values).astype(np.float32), dtype=tf.int32)
print()

print()
model.fit([player_names_padded], 
          target_player_names,
          batch_size=288, 
          epochs=10, 
          validation_split=0.2)"""

[791 543 481 ... 367 679 293]


Epoch 1/10


InvalidArgumentError: Graph execution error:

Detected at node 'sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits' defined at (most recent call last):
    File "<frozen runpy>", line 198, in _run_module_as_main
    File "<frozen runpy>", line 88, in _run_code
    File "C:\Python-3.11.4\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\Python-3.11.4\Lib\site-packages\traitlets\config\application.py", line 1043, in launch_instance
      app.start()
    File "C:\Python-3.11.4\Lib\site-packages\ipykernel\kernelapp.py", line 736, in start
      self.io_loop.start()
    File "C:\Python-3.11.4\Lib\site-packages\tornado\platform\asyncio.py", line 195, in start
      self.asyncio_loop.run_forever()
    File "C:\Python-3.11.4\Lib\asyncio\base_events.py", line 607, in run_forever
      self._run_once()
    File "C:\Python-3.11.4\Lib\asyncio\base_events.py", line 1922, in _run_once
     

In [8]:
from tensorflow.keras.layers import Input, Embedding, concatenate, Dropout, Dense, GlobalAveragePooling1D, Reshape, Flatten

player_input_dim = len(player_le.classes_)
team_input_dim = len(team_le.classes_)
season_input_dim = len(season_le.classes_)

# Define input layers
player_input = Input(shape=(9,), dtype='int32', name='player_input')
team_input = Input(shape=(2,), dtype='int32', name='team_input')
season_input = Input(shape=(1,), dtype='int32', name='season_input')

# Embedding layers
player_embedding = Embedding(input_dim=player_input_dim + 1, output_dim=64)(player_input)
team_embedding = Embedding(input_dim=team_input_dim + 1, output_dim=64)(team_input)
season_embedding = Embedding(input_dim=season_input_dim + 1, output_dim=64)(season_input)

# Reshape team and season embeddings to make their shapes compatible with player_embedding for concatenation
team_embedding_reshaped = Reshape((1, -1))(team_embedding)  # Reshape to (None, 1, 128)
season_embedding_reshaped = Reshape((1, -1))(season_embedding)  # Reshape to (None, 1, 64)

# Apply pooling or flattening to the player embeddings
player_embedding_pooled = GlobalAveragePooling1D()(player_embedding)

# Since now player_embedding_pooled is 2D (None, 64), we need to adjust team and season embeddings as well
team_embedding_flat = Flatten()(team_embedding_reshaped)
season_embedding_flat = Flatten()(season_embedding_reshaped)

# Combine embeddings
combined_embedding = concatenate([player_embedding_pooled, team_embedding_flat, season_embedding_flat])

# Fully connected layers to process the combined embeddings
dense_layer = Dense(128, activation='relu')(combined_embedding)
dense_layer = Dropout(0.5)(dense_layer)  # Adding dropout for regularization
output_layer = Dense(player_input_dim, activation='softmax')(dense_layer)  # Output layer for classification

# Create and compile the model
model = Model(inputs=[player_input, team_input, season_input], outputs=output_layer)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [11]:
# Ensure it's padded to have a fixed length if it's not already
from tensorflow.keras.preprocessing.sequence import pad_sequences
player_names_padded = pad_sequences(X_train['player_names'], maxlen=9, padding='post', value=0)
print(X_train['season'])
team_home = X_train['home_team']
team_away = X_train['away_team']
season = X_train['season']

# Directly specify dtype when using np.stack to avoid type-related issues
team_tensor = np.stack((team_home.values, team_away.values), axis=-1).astype('int32')

# Use TensorFlow operations to ensure compatibility if the above doesn't resolve the issue
team_tensor = tf.cast(team_tensor, dtype=tf.int32)

season_tensor = tf.cast(X_train['season'], dtype=tf.int32)
print(X_train['target_player_name'].values)
target_player_names = tf.cast(np.array(X_train['target_player_name'].values).astype(np.float32), dtype=tf.int32)
print()
print([player_names_padded, season_tensor, team_tensor])
print()
model.fit([player_names_padded, season_tensor, team_tensor], 
          target_player_names,
          batch_size=32, 
          epochs=10, 
          validation_split=0.2)

120762    4
80136     2
104090    3
24772     0
43933     1
         ..
58268     2
129110    4
69783     2
56845     2
42652     1
Name: season, Length: 123625, dtype: int64
[546 314 180 ... 616 725 191]

[array([[ 53, 413, 596, ..., 467, 594, 641],
       [182, 306, 543, ..., 624, 644, 681],
       [ 22,  33, 503, ...,  89, 481, 546],
       ...,
       [281, 313, 495, ..., 279, 367, 720],
       [ 49, 338, 451, ..., 458, 592, 679],
       [ 36, 340, 404, ..., 293, 348, 755]]), <tf.Tensor: shape=(123625,), dtype=int32, numpy=array([4, 2, 3, ..., 2, 2, 1])>, <tf.Tensor: shape=(123625, 2), dtype=int32, numpy=
array([[16, 31],
       [13, 18],
       [23, 25],
       ...,
       [ 1, 26],
       [ 8, 21],
       [29,  1]])>]

Epoch 1/10


InvalidArgumentError: Graph execution error:

Detected at node 'model/embedding_2/embedding_lookup' defined at (most recent call last):
    File "<frozen runpy>", line 198, in _run_module_as_main
    File "<frozen runpy>", line 88, in _run_code
    File "C:\Python-3.11.4\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\Python-3.11.4\Lib\site-packages\traitlets\config\application.py", line 1043, in launch_instance
      app.start()
    File "C:\Python-3.11.4\Lib\site-packages\ipykernel\kernelapp.py", line 736, in start
      self.io_loop.start()
    File "C:\Python-3.11.4\Lib\site-packages\tornado\platform\asyncio.py", line 195, in start
      self.asyncio_loop.run_forever()
    File "C:\Python-3.11.4\Lib\asyncio\base_events.py", line 607, in run_forever
      self._run_once()
    File "C:\Python-3.11.4\Lib\asyncio\base_events.py", line 1922, in _run_once
      handle._run()
    File "C:\Python-3.11.4\Lib\asyncio\events.py", lin

In [13]:
"""# OTHER
import tensorflow_recommenders as tfrs
from typing import Dict, Text

class NBARecommendationModel(tfrs.models.Model):

    def __init__(self, unique_player_names, unique_teams, unique_seasons):
        super().__init__()
        embedding_dimension = 32

        # Player model for candidates
        self.player_model = tf.keras.Sequential([
            tf.keras.layers.StringLookup(
                vocabulary=unique_player_names, mask_token=None),
            tf.keras.layers.Embedding(len(unique_player_names) + 1, embedding_dimension)
        ])

        # Player embedding for query
        self.query_player_embedding = tf.keras.Sequential([
            tf.keras.layers.StringLookup(
                vocabulary=unique_player_names, mask_token=None),
            tf.keras.layers.Embedding(len(unique_player_names) + 1, embedding_dimension)
        ])

        # Season embedding
        self.season_embedding = tf.keras.Sequential([
            tf.keras.layers.StringLookup(
                vocabulary=unique_seasons, mask_token=None),
            tf.keras.layers.Embedding(len(unique_seasons) + 1, embedding_dimension)
        ])

        # Team embedding
        self.team_embedding = tf.keras.Sequential([
            tf.keras.layers.StringLookup(
                vocabulary=unique_teams, mask_token=None),
            tf.keras.layers.Embedding(len(unique_teams) + 1, embedding_dimension)
        ])

        # Query model
        self.query_model = tf.keras.Sequential([
          tf.keras.layers.Dense(64, activation='relu'),
          tf.keras.layers.Dense(embedding_dimension),
        ])

        # Retrieval task
        self.task = tfrs.tasks.Retrieval(
            metrics=tfrs.metrics.FactorizedTopK(
                candidates=self.player_model.embeddings
            )
        )

    def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
        # Embedding for season, home_team, and away_team
        season_embeddings = self.season_embedding(features["season"])
        home_team_embeddings = self.team_embedding(features["home_team"])
        away_team_embeddings = self.team_embedding(features["away_team"])

        # Embeddings for the 9 known players
        player_embeddings = self.query_player_embedding(features["player_names"])

        # Combine all embeddings
        combined_embeddings = tf.concat([
            season_embeddings, home_team_embeddings, away_team_embeddings, tf.reduce_mean(player_embeddings, axis=1)
        ], axis=1)

        user_embeddings = self.query_model(combined_embeddings)

        # Candidate player embeddings
        positive_player_embeddings = self.player_model(features["target_player_name"])

        # The task computes the loss and the metrics.
        return self.task(user_embeddings, positive_player_embeddings)"""

In [14]:
"""# OTHER
model = NBARecommendationModel(unique_player_names=unique_player_names, unique_teams=unique_teams, unique_seasons=unique_seasons)

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

# Fit the model
model.fit(tf_dataset, epochs=10)"""

C:\Python-3.11.4\Lib\site-packages\numpy\core\numeric.py:2468: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return bool(asarray(a1 == a2).all())


ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type int).